In [145]:
import numpy as np
import time
from sklearn.decomposition import PCA, SparsePCA
from scipy.linalg import svd

In [489]:
def piecewise(val,cond):
    return val * cond

def scad(vec, lam, a=3.7):
    return np.multiply(np.multiply(vec - lam * np.sign(vec), np.abs(vec) > lam), np.abs(vec) < (2*lam)) + \
            np.multiply(vec, np.abs(vec) > a * lam) + \
            np.multiply(np.multiply(((a - 1) * vec - np.sign(vec) * lam * a )/ (a - 2), \
                                    np.abs(vec) > (2*lam)), np.abs(vec) < (a*lam))

def l2_project(beta):
    if np.linalg.norm(beta) > 0:
        return beta/np.linalg.norm(beta)
    else:
        return beta

def scad_obj(vec, lamb, a=3.7):
    piece1 = piecewise(lamb*np.abs(vec),np.abs(vec)<=lamb)
    piece2 = piecewise(1/(2*(a-1))*(2*a*lamb*np.abs(vec)-vec**2-lamb**2),np.logical_and(lamb<np.abs(vec),np.abs(vec)<=a*lamb))
    piece3 = piecewise(lamb**2*(a+1)/2,a*lamb<np.abs(vec))
    return np.sum(piece1+piece2+piece3)

# Gradient of pca objective ||Sigma-beta beta^T||_F^2
def PCA_grad(Sigma,beta):
    return 4*(-np.dot(Sigma,beta)+np.sum(np.square(beta))*beta)

# SCAD = L1 - cvx
# This is the gradient of the -cvx part
def SCAD_ccv_grad(beta,lamb,a=3.7):
    piece1 = piecewise(0,np.abs(beta)<=lamb)
    piece2 = piecewise(1/(a-1)*(beta-lamb*np.sign(beta)),np.logical_and(lamb<np.abs(beta),np.abs(beta)<=a*lamb))
    piece3 = piecewise(lamb*np.sign(beta),np.abs(beta)>a*lamb)
    return -(piece1+piece2+piece3)

# Proximal operator for L1
def prox_l1(v,c):
    return np.sign(v) * np.maximum(np.abs(v)-c,0)

def PCA_obj(Sigma,beta):
    return np.linalg.norm(Sigma-np.outer(beta,beta),ord="fro")**2

def l1_obj(beta,lamb):
    return lamb*np.sum(np.abs(beta))

# This is the objective function for SCAD
def full_scad_obj(Sigma,beta,lamb,a=3.7):
    return PCA_obj(Sigma,beta)+scad_obj(beta,lamb,a)

# def prox_line_search(f,x,lamb,beta=.5):
#    while f()

In [428]:
# Squared error between both beta1 and beta2 and the normalized versions
def sq_error(beta1,beta2):
    return np.linalg.norm(beta1-beta2), np.linalg.norm(l2_project(beta1)-l2_project(beta2))

# Spiked covariance

In [490]:
# Generates n samples from spiked covariance model of dimension d, spike at u with signal strength gamma
def spiked_cov(n,d,u,gamma=1,return_cov=False):
    u = u / np.linalg.norm(u)
    cov = np.eye(d) + gamma * np.outer(u,u)
    X = np.random.multivariate_normal(np.zeros(d),cov,size=n)
    if return_cov:
        return X, cov
    else:
        return X

# First s entries are 1
def equal_spike(n,d,s,gamma=1,return_cov=False):
    u = np.zeros(d)
    u[:s] = 1
    return spiked_cov(n,d,u,gamma,return_cov)

# First entry is alpha, next s-1 entries are 1
def heavy_spike(n,d,s,gamma=1,alpha=10,return_cov=False):
    u = np.zeros(d)
    u[0] = alpha
    u[1:s] = 1
    return spiked_cov(n,d,u,gamma,return_cov)

In [501]:
n = 200
d = 10
s = 3
X, pop_cov = equal_spike(n,d,s,return_cov=True)
Sigma = 1/n*np.dot(X.T,X)

In [502]:
sample_vals, sample_vecs = np.linalg.eigh(Sigma)
sample_vecs[:,d-1]*sample_vals[d-1]

array([-1.27390522, -1.3683025 , -0.82566492, -0.4867676 ,  0.43420108,
        0.05288773,  0.05499859,  0.02648846,  0.00445584,  0.00904706])

In [503]:
pop_vals, pop_vecs = np.linalg.eigh(pop_cov)
pop_vecs[:,d-1]*pop_vals[d-1]

array([-1.15470054, -1.15470054, -1.15470054,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [504]:
pop_vecs[:,d-1]

array([-0.57735027, -0.57735027, -0.57735027,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [505]:
beta_star = pop_vecs[:,d-1]*pop_vals[d-1]
beta_star_normalized = pop_vecs[:,d-1]

In [506]:
start = time.time()
spca = SparsePCA(n_components=1, method='cd',normalize_components=True)
SPCAResult = spca.fit(X)
spca_time_result = time.time() - start
SPCAResult.components_

array([[-0.61796707, -0.65560347, -0.36772513, -0.17958067,  0.14432533,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

# Useful functions

In [480]:
# Gradient of pca objective ||Sigma-beta beta^T||_F^2
def PCA_grad(Sigma,beta):
    return 4*(-np.dot(Sigma,beta)+np.sum(np.square(beta))*beta)

# SCAD = L1 - cvx
# This is the gradient of the -cvx part
def SCAD_ccv_grad(beta,lamb,a=3.7):
    piece1 = piecewise(0,np.abs(beta)<=lamb)
    piece2 = piecewise(1/(a-1)*(beta-lamb*np.sign(beta)),np.logical_and(lamb<np.abs(beta),np.abs(beta)<=a*lamb))
    piece3 = piecewise(lamb*np.sign(beta),np.abs(beta)>a*lamb)
    return -(piece1+piece2+piece3)

# Proximal operator for L1
def prox_l1(v,c):
    return np.sign(v) * np.maximum(np.abs(v)-c,0)

# SCAD subgradient descent

In [507]:
lr = 0.005
lamb = 1
(U, s, Vh) = svd(X)
beta = Vh[0]
np.random.seed(70)
beta=np.random.normal(size=d)
for i in range(10000):
    beta = beta - lr/np.sqrt(i+1) * (PCA_grad(Sigma,beta)+SCAD_ccv_grad(beta,lamb)+lamb*np.sign(beta))
    # beta = scad(beta,lr * lamb)
    #beta = beta + 0.001 * np.dot(Sigma, beta)
    #beta=beta/np.linalg.norm(beta)
print(beta)
print(sq_error(beta,beta_star))
print(full_scad_obj(Sigma,beta,lamb))

[-7.67821789e-06 -4.10170756e-06  1.04323628e-06 -1.91387237e-05
 -3.90126780e-05 -2.34557199e-05 -8.97173383e-01 -3.01305697e-05
 -2.46649797e-05 -1.72555075e-05]
(2.1920071369314207, 1.4142086767594086)
13.549835497302015


# Naive prox with scad

In [508]:
lr = 0.001
lamb = 1
(U, s, Vh) = svd(X)
beta = Vh[0]
np.random.seed(70)
beta=np.random.normal(size=d)
for i in range(10000):
    beta = beta - lr * PCA_grad(Sigma,beta)
    beta = scad(beta,lr * lamb)
    #beta = beta + 0.001 * np.dot(Sigma, beta)
    #beta=beta/np.linalg.norm(beta)
print(beta)
print(sq_error(beta,beta_star))
print(full_scad_obj(Sigma,beta,lamb))

[ 0.87450708  0.92941139  0.56265027  0.33029671 -0.30038484  0.
 -0.03952788  0.          0.         -0.01009864]
(3.4075638345251127, 1.9660489586034235)
12.20212925294048


# Lasso via prox

In [509]:
lr = 0.001
lamb = 1
(U, s, Vh) = svd(X)
beta = Vh[0]
np.random.seed(70)
beta = np.random.normal(size=d)
for i in range(10000):
    # print(SCAD_ccv_grad(beta,lamb))
    beta = beta - lr * (PCA_grad(Sigma,beta))
    beta = prox_l1(beta,lr * lamb)
    #beta = beta + 0.001 * np.dot(Sigma, beta)
    #beta=beta/np.linalg.norm(beta)
print(beta)
print(sq_error(beta,beta_star))
print(full_scad_obj(Sigma,beta,lamb))

[ 0.          0.         -0.          0.         -0.          0.
 -0.90224557 -0.         -0.          0.        ]
(2.1940936764173657, 1.4142135623730951)
13.549597590232448


# Prox by splitting scad

In [510]:
lr = 0.001
lamb = 1
(U, s, Vh) = svd(X)
beta = Vh[0]
np.random.seed(70)
beta = np.random.normal(size=d)
for i in range(10000):
    # print(SCAD_ccv_grad(beta,lamb))
    beta = beta - lr * (PCA_grad(Sigma,beta)+SCAD_ccv_grad(beta,lamb,a=3.7))
    beta = prox_l1(beta,lr * lamb)
    #beta = beta + 0.001 * np.dot(Sigma, beta)
    #beta=beta/np.linalg.norm(beta)
print(beta)
print(sq_error(beta,beta_star))
print(full_scad_obj(Sigma,beta,lamb))

[ 0.          0.         -0.          0.         -0.          0.
 -0.90224557 -0.         -0.          0.        ]
(2.1940936764173657, 1.4142135623730951)
13.549597590232448


In [335]:
l2_project(beta)

array([ 0.99995717,  0.00850883,  0.        , -0.        ,  0.        ,
        0.        , -0.        , -0.        , -0.        ,  0.00364157])

In [362]:
PCA_grad(Sigma,beta)+SCAD_ccv_grad(beta,lamb,a=3.7)+lamb*np.sign(beta)

array([-1.55431223e-15, -5.55111512e-17, -1.77692011e-01,  7.24407440e-02,
       -2.29567260e-01, -2.89558597e-01,  1.46486986e-01,  4.79587529e-02,
        5.68151406e-02,  0.00000000e+00])

In [366]:
PCA_grad(Sigma,beta)+lamb*np.sign(beta)

array([-1.09912079e-14, -5.55111512e-17, -1.74526818e-01,  7.09576265e-02,
       -2.25288305e-01, -2.84212705e-01,  1.43867883e-01,  4.68470857e-02,
        5.61566606e-02,  0.00000000e+00])

# Prox-MCP

In [435]:
def MCP_ccv_grad(beta,lamb,a=3.7):
    piece1 = piecewise(beta/a,np.abs(beta)<=a*lamb)
    piece2 = piecewise(lamb*np.sign(beta),a*lamb<np.abs(beta))
    return -(piece1+piece2)

In [467]:
lr = 0.001
lamb = 1
(U, s, Vh) = svd(X)
beta = Vh[0]
np.random.seed(70)
beta = np.random.normal(size=d)
for i in range(1000):
    # print(SCAD_ccv_grad(beta,lamb))
    beta = beta - lr * (PCA_grad(Sigma,beta)+MCP_ccv_grad(beta,lamb))
    beta = prox(beta,lr * lamb)
    #beta = beta + 0.001 * np.dot(Sigma, beta)
    #beta=beta/np.linalg.norm(beta)
print(beta)
print(sq_error(beta,beta_star))
print(full_scad_obj(Sigma,beta,lamb))

[-0.00535729 -0.         -0.61348257 -0.15478699  0.         -0.
  0.          0.         -0.10489682 -0.          0.         -0.
 -0.          0.110186   -0.28179378 -0.          0.07998022  0.
 -0.03674899 -0.         -0.         -0.         -0.         -0.13830803
 -0.00389978 -0.          0.         -0.1771918  -0.         -0.
 -0.08442501  0.4784156   0.         -0.          0.          0.
 -0.         -0.          0.0153236   0.03333093  0.          0.
  0.23405114 -0.          0.         -0.          0.         -0.06245266
  0.          0.20226264  0.          0.          0.40387902 -0.
 -0.04598196 -0.          0.03784555 -0.         -0.4877225  -0.02093607
  0.          0.         -0.17424607  0.01187938  0.         -0.
 -0.03523897 -0.         -0.          0.          0.          0.25485659
  0.07739468  0.         -0.          0.          0.         -0.40503046
 -0.         -0.          0.         -0.         -0.          0.
  0.08065139 -0.          0.         -0.04719326  

In [327]:
l2_project(beta)

array([ 9.99983578e-01,  5.68058341e-03,  0.00000000e+00, -0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
       -0.00000000e+00,  7.58097185e-04])

# Misc: Sparse linear regression with scad

In [367]:
X = np.random.multivariate_normal(np.zeros(6),np.eye(6),size=500)
beta_star = np.array([1,2,3,0,0,0])
Y = np.dot(X,beta_star) + np.random.normal(scale=.2,size=500)

In [368]:
# LASSO
lr = 0.01
lamb = 0.25
samp_cov = np.dot(X.T,X)
xty = np.dot(X.T,Y)
beta = np.array([0.1,-0.1,0.2,0.1,0.3,-0.4])
for i in range(10000):
    beta = beta - lr * 1/500 *(np.dot(samp_cov,beta)-xty)
    beta = prox_l1(beta,lr * lamb)

In [369]:
beta

array([0.74472985, 1.7636387 , 2.7332998 , 0.        , 0.        ,
       0.        ])

In [381]:
# Splitting SCAD
lr = 0.01
lamb = .4
samp_cov = np.dot(X.T,X)
xty = np.dot(X.T,Y)
beta = np.array([0.1,-0.1,0.2,0.1,0.3,-0.4])
for i in range(10000):
    beta = beta - lr * 1/500 *(np.dot(samp_cov,beta)-xty)-lr*SCAD_ccv_grad(beta,lamb)
    beta = prox_l1(beta,lr * lamb)
beta

array([0.72265217, 1.99978766, 2.98029579, 0.        , 0.        ,
       0.        ])

In [244]:
1/500 *(np.dot(samp_cov,beta)-xty)+lamb*np.sign(beta)+SCAD_ccv_grad(beta,lamb)

array([ 3.32622818e-13, -1.10778053e-12, -2.22005747e-12, -1.08399197e-02,
        1.99455401e-03,  1.91817508e-02])

In [245]:
beta_scad1 = beta

In [377]:
# Prox gradient with scad, this actually works for debiasing (but check the scad penalty?)
lr = 0.01
lamb = 3
samp_cov = np.dot(X.T,X)
xty = np.dot(X.T,Y)
beta = np.array([0.1,-0.1,0.2,0.1,0.3,-0.4])
for i in range(10000):
    beta = beta - lr * 1/500 *(np.dot(samp_cov,beta)-xty)
    beta = scad(beta,lr * lamb)
beta

array([0.98852003, 0.        , 3.08700832, 0.        , 0.        ,
       0.        ])

In [247]:
1/500 *(np.dot(samp_cov,beta)-xty)+lamb*np.sign(beta)+SCAD_ccv_grad(beta,lamb)

array([ 3.15775310e-01, -2.20379270e-14, -3.63598041e-14, -7.60001987e-03,
        5.00000000e-01, -7.88898215e-03])